# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **Taipei, Taiwan**.

Since there are lots of restaurants in Taipei we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Taipei center will be obtained using **Google Maps API geocoding** of well known Taipei location (Taipei Main Station)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Taipei city center.

Let's first find the latitude & longitude of Taipei city center, using specific, well known address and Google Maps geocoding API.

In [1]:
# Google API Key
google_api_key = 'your Google API Key'

In [2]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Taipei Main Station, Taipei, Taiwan'
city_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, city_center))

Coordinate of Taipei Main Station, Taipei, Taiwan: [25.0471778, 121.5172047]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Taipei Main Station. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [3]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=51, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=51, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Taipei center longitude={}, latitude={}'.format(city_center[1], city_center[0]))
x, y = lonlat_to_xy(city_center[1], city_center[0])
print('Taipei center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Taipei center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Taipei center longitude=121.5172047, latitude=25.0471778
Taipei center UTM X=350417.95463453245, Y=2770991.266804541
Taipei center longitude=121.5172047, latitude=25.047177799999993


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [4]:
city_center_x, city_center_y = lonlat_to_xy(city_center[1], city_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2            # Vertical offset for hexagonal grid cells
x_min = city_center_x - 6000
x_step = 600
y_min = city_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(city_center_x, city_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: Taiepi center location and candidate neighborhood centers:

In [5]:
#!pip install folium

import folium

In [6]:
map_city = folium.Map(location=city_center, zoom_start=13)
folium.Marker(city_center, popup='Taipei Main Station').add_to(map_city)
for lat, lon in zip(latitudes, longitudes):
#     folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_city) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_city)
#     folium.Marker([lat, lon]).add_to(map_city)
map_city

<img src="./9-3-2.jpg">

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Taipei Main Station. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [7]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, city_center[0], city_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(city_center[0], city_center[1], addr))

Reverse geocoding check
-----------------------
Address of [25.0471778, 121.5172047] is: Taipei station, No. 3, Beiping West Road, Zhongzheng District, Taipei City, Taiwan 100


In [67]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Taiwan', '') # We don't need country part of address
    addresses.append(address)
#     print('\n', address)
    print(' .', end='')
print('\n', ' -- done. --')

Obtaining location addresses: 
 No. 2, Lane 121, Yuantong Road, Zhonghe District, New Taipei City 235

 No. 8, Mingren Street, Zhonghe District, New Taipei City 235

 No. 4, Alley 6, Lane 106, Baojian Road, Zhonghe District, New Taipei City 235

 No. 98, Dayong Street, Zhonghe District, New Taipei City 235

 No. 12, Lane 15, Zili Road, Zhonghe District, New Taipei City 235

 No. 127, Section 4, Huanhe East Road, Yonghe District, New Taipei City 234

 No. 124, Jingfu Street, Wenshan District, Taipei City 116

 No. 213, Jianba Road, Zhonghe District, New Taipei City 235

 No. 87號, Liancheng Road, Zhonghe District, New Taipei City 235

 No. 167, Lane 165, Zhonghe Road, Zhonghe District, New Taipei City 235

 No. 387, Zhonghe Road, Yonghe District, New Taipei City 234

 No. 24, Lane 154, Anle Road, Yonghe District, New Taipei City 234

 No. 16, Alley 7, Lane 17, Yongyuan Road, Yonghe District, New Taipei City 234

 No. 36, Section 2, Chenggong Road, Yonghe District, New Taipei City 234

 U


 No. 11, Lane 2, Yongkang Street, Da’an District, Taipei City 106

 Daan Park Station, No. 100, Section 3, Xinyi Road, Da’an District, Taipei City 106

 No. 160, Section 3, Xinyi Road, Da’an District, Taipei City 106

 No. 97, Section 4, Xinyi Road, Da’an District, Taipei City 106

 No. 235, Section 4, Xinyi Road, Da’an District, Taipei City 106

 No. 413-1, Section 4, Xinyi Road, Xinyi District, Taipei City 110

 Taipei 101, No. 7, Section 5, Xinyi Road, Xinyi District, Taipei City 110

 No. 7, Lane 69,Songyong Rd, Xinyi District, Taipei City 110

 Dahan Bridge, Xinzhuang District, New Taipei City 242

 220, New Taipei City, Banqiao District, 機慢車專用道

 220, New Taipei City, Banqiao District, 機慢車專用道

 Section 4, Huanhe West Road, Banqiao District, New Taipei City 220

 108, Taipei City, Wanhua District, 機慢車專用道

 108, Taipei City, Wanhua District, Unnamed Road

 No. 73, Section 2, Huanhe South Road, Wanhua District, Taipei City 108

 Taiwan, 台北市萬華區

 No. 135-2, Yanping South Road, Zhong


 Unnamed Road, Neihu District, Taipei City 114

 No. 458號, Shuhong West Road, Sanchong District, New Taipei City 241

 Water Park, Sanchong District, New Taipei City 241

 No. 10, Dayou Street, Sanchong District, New Taipei City 241

 241, New Taipei City, Sanchong District, 重陽路二段52巷75弄1號

 Datong Park [New Taipei City Bus], Sanchong District, New Taipei City 241

 No. 158, Section 1, Ziqiang Road, Sanchong District, New Taipei City 241

 No. 77, Fude North Road, Sanchong District, New Taipei City 241

 No. 87, Section 2, Huanhe North Road, Datong District, Taipei City 103

 No. 136號, Changji Street, Datong District, Taipei City 103

 No. 15, Lane 41, Fushun Street, Zhongshan District, Taipei City 10491

 No. 30-4, Lane 68, Section 3, Xinsheng North Road, Zhongshan District, Taipei City 10491

 No. 7, Lane 170, Dehui Street, Zhongshan District, Taipei City 10491

 No. 79, Lane 356, Longjiang Road, Zhongshan District, Taipei City 10491

 No. 49, Alley 13, Lane 512, Minzu East Road, Zho

In [69]:
# addresses[150:170]
len(addresses)

364

Looking good. Let's now place all this into a Pandas dataframe.

In [68]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"No. 2, Lane 121, Yuantong Road, Zhonghe Distri...",24.995397,121.499994,348617.954635,2.765275e+06,5992.495307
1,"No. 8, Mingren Street, Zhonghe District, New T...",24.995457,121.505937,349217.954635,2.765275e+06,5840.376700
2,"No. 4, Alley 6, Lane 106, Baojian Road, Zhongh...",24.995516,121.511881,349817.954635,2.765275e+06,5747.173218
3,"No. 98, Dayong Street, Zhonghe District, New T...",24.995576,121.517825,350417.954635,2.765275e+06,5715.767665
4,"No. 12, Lane 15, Zili Road, Zhonghe District, ...",24.995635,121.523768,351017.954635,2.765275e+06,5747.173218
5,"No. 127, Section 4, Huanhe East Road, Yonghe D...",24.995694,121.529712,351617.954635,2.765275e+06,5840.376700
6,"No. 124, Jingfu Street, Wenshan District, Taip...",24.995752,121.535656,352217.954635,2.765275e+06,5992.495307
7,"No. 213, Jianba Road, Zhonghe District, New Ta...",24.999998,121.491021,347717.954635,2.765795e+06,5855.766389
8,"No. 87號, Liancheng Road, Zhonghe District, New...",25.000058,121.496965,348317.954635,2.765795e+06,5604.462508
9,"No. 167, Lane 165, Zhonghe Road, Zhonghe Distr...",25.000118,121.502909,348917.954635,2.765795e+06,5408.326913


...and let's now save/persist this data into local file.

In [81]:
df_locations.to_pickle('locations.pkl')     

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on Italian restaurants in the neighborhood.

Foursquare credentials are defined in hidden cell bellow.

In [90]:
# Foursquare ID/Secret/Foursquare API version
client_id = 'your Foursquare ID' # your Foursquare ID
client_secret = 'your Foursquare Secret' # your Foursquare Secret
version = 'your Foursquare API version' # your Foursquare API version

In [83]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', 台灣省', '')
    address = address.replace(', 台灣', '')
    address = address.replace(', Taiwan', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [87]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, limit=100)
        print('\n There are ', len(venues), ' venues for (', lat, ',', lon, ') to be selected.')
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            print('\n', is_res, ' ', is_italian, ' ', venue)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
    print('\n', ' -- done. --')
    return restaurants, italian_restaurants, location_restaurants



In [91]:
# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [92]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1262
Total number of Italian restaurants: 61
Percentage of Italian restaurants: 4.83%
Average number of restaurants in neighborhood: 2.9615384615384617


In [93]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4edb48c34901c8be1ea30507', '泰國菜', 24.994249713881725, 121.49975390265546, '臺灣', 129, False, 348592.35389252775, 2765148.7080648555)
('4bee90763686c9b672a0246e', '五花馬水餃館', 24.998199, 121.5000937, '235台湾國台北縣中和市景平路495號, 中和市,  235, 臺灣', 312, False, 348631.49148220033, 2765585.7241906556)
('4d20144b5acaa35db20fc535', '炒翻天餐飲店', 24.996419723832847, 121.50307782549402, '新北市中和區南華路36號, 235, 臺灣', 331, False, 348930.51367471134, 2765385.3344932194)
('563ec56ccd108d4763aa127b', '港麗港式茶餐廳', 24.994414, 121.504985, '臺灣', 150, False, 349120.56439614156, 2765161.0704014087)
('4e1691dfd4c0c7a8fbb3b1e9', '鬍鬚張', 24.994702679542083, 121.50530301538814, '景安路128號, 臺灣省, 臺灣省, 臺灣', 105, False, 349153.0162619776, 2765192.688555303)
('4f78466de4b067aa304e749e', 'sushi take-out (爭鮮外帶)', 24.99380714640073, 121.50484013482638, '臺灣', 214, False, 349105.2009737861, 2765094.0208590967)
('4d42c1d57fb05481251e6e79', '大埔鐵板燒', 24.997524, 121.506586, '景安路52號, 新北市, 臺灣', 239, Fa

In [94]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('4daefce70f2ca3443cc7e26d', '海盜船義式料理', 25.007450566901777, 121.51305913925171, '永和路一段65巷5-1號, 臺灣省, 臺灣省, 臺灣', 291, True, 349951.3787680799, 2766595.9428502657)
('552e6362498e772579f25a35', 'Saizeriya 薩莉亞', 25.012303054526733, 121.51497288853705, '永和路二段121號 (頂溪店), 臺灣', 348, True, 350150.415348767, 2767131.2513037426)
('518a4bff498ea1a34ff8934a', '嵐迪義大利麵館 Landi Pasta', 25.01502978967108, 121.51544130460657, '臺灣', 230, True, 350200.9979794085, 2767432.7268443564)
('5249019a498e6a3375bbfbe7', 'Just Italian (義饗食堂)', 25.0132398443613, 121.53599366193363, '羅斯福路四段83號二樓, 台北市, 臺灣', 152, True, 352272.9798533323, 2767211.913861379)
('53d3a127498e45540f53ad2c', '紅蔥 Shallot', 25.016149, 121.543215, '基隆路三段137號, 台北市, 臺灣', 255, True, 353005.2227769593, 2767526.2525170096)
('5749643c498e3b6d1149ca5c', '義式傳奇', 25.02027, 121.49562, '萬華區台北市萬華區萬大路486巷59弄4號, 台北市,  108, 臺灣', 196, True, 348207.08491504844, 2768035.1667253007)
('534690a4498e30a8bf88a312', 

In [19]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 
Restaurants around location 102: 
Restaurants around location 103: 
Restaurants around location 104: 新東南海鮮餐廳, 山內雞肉, 臭老闆蒸臭豆腐, 來來水餃館, 秀昌水餃牛肉麵, 林家豬腸冬粉, 蒸臭豆腐
Restaurants around location 105: 秘魯烤雞 Polleria, Ebisu Curry & Coffee, 蔡萬興寧波餐廳, 黃龍莊, 四海包子店, 牯嶺油飯
Restaurants around location 106: 晉江茶堂, 蘇杭點心店, 樂埔町 Leputing, TGI Fridays (星期五美式餐廳), Buffalo 水牛城美式碳烤牛排, Sukiya (すき家 Sukiya)
Restaurants around location 107: 台北月見ル君想フ, 游壽司, 全國素食自助餐, 台電勵進餐廳 Taipower Restaurant, 林記燈籠牛肉麵
Restaurants around location 108: 鴻疆石, 大隱酒食 James Kitchen, 初魚料亭
Restaurants around location 109: 樂子 The Diner, 耍性子 Xai Xing Dé, Teotihuacán, 日日香小館, Ruei An Taiwan Buffet (瑞安自助餐), 上鼎豐小籠湯包
Restaurants around location 110: 禾豐涮涮鍋, 小李子清粥小菜, 一流清粥小菜, 七里亭, 爭鮮迴轉壽司 Sushi Express 科技店, 角屋關東煮, 無名子清粥小菜


Let's now see all the collected restaurants in our area of interest on map, and let's also show Italian restaurants in different color.

In [20]:
map_city = folium.Map(location=city_center, zoom_start=13)
folium.Marker(city_center, popup='Taipei Main Station').add_to(map_city)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_city)
map_city

<img src="./9-3-2-2.jpg">

Looking good. So now we have all the restaurants in area within few kilometers from Taipei Main Station, and we know which ones are Italian restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Taipei that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 6km from city center** (Taipei Main Station). We have also **identified Italian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Taipei - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without Italian restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [21]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 2.92032967032967


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"No. 2, Lane 121, Yuantong Road, Zhonghe Distri...",24.995397,121.499994,348617.954635,2.765275e+06,5992.495307,1
1,"No. 8, Mingren Street, Zhonghe District, New T...",24.995457,121.505937,349217.954635,2.765275e+06,5840.376700,4
2,"No. 4, Alley 6, Lane 106, Baojian Road, Zhongh...",24.995516,121.511881,349817.954635,2.765275e+06,5747.173218,1
3,"No. 98, Dayong Street, Zhonghe District, New T...",24.995576,121.517825,350417.954635,2.765275e+06,5715.767665,1
4,"No. 12, Lane 15, Zili Road, Zhonghe District, ...",24.995635,121.523768,351017.954635,2.765275e+06,5747.173218,3
5,"No. 127, Section 4, Huanhe East Road, Yonghe D...",24.995694,121.529712,351617.954635,2.765275e+06,5840.376700,0
6,"No. 124, Jingfu Street, Wenshan District, Taip...",24.995752,121.535656,352217.954635,2.765275e+06,5992.495307,1
7,"No. 213, Jianba Road, Zhonghe District, New Ta...",24.999998,121.491021,347717.954635,2.765795e+06,5855.766389,0
8,"No. 87號, Liancheng Road, Zhonghe District, New...",25.000058,121.496965,348317.954635,2.765795e+06,5604.462508,2
9,"No. 167, Lane 165, Zhonghe Road, Zhonghe Distr...",25.000118,121.502909,348917.954635,2.765795e+06,5408.326913,4


OK, now let's calculate the **distance to nearest Italian restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [22]:
distances_to_italian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in italian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_italian_restaurant.append(min_distance)

df_locations['Distance to Italian restaurant'] = distances_to_italian_restaurant

In [23]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Italian restaurant
0,"No. 2, Lane 121, Yuantong Road, Zhonghe Distri...",24.995397,121.499994,348617.954635,2.765275e+06,5992.495307,1,1876.590396
1,"No. 8, Mingren Street, Zhonghe District, New T...",24.995457,121.505937,349217.954635,2.765275e+06,5840.376700,4,1510.457730
2,"No. 4, Alley 6, Lane 106, Baojian Road, Zhongh...",24.995516,121.511881,349817.954635,2.765275e+06,5747.173218,1,1327.167507
3,"No. 98, Dayong Street, Zhonghe District, New T...",24.995576,121.517825,350417.954635,2.765275e+06,5715.767665,1,1400.451582
4,"No. 12, Lane 15, Zili Road, Zhonghe District, ...",24.995635,121.523768,351017.954635,2.765275e+06,5747.173218,3,1697.396734
5,"No. 127, Section 4, Huanhe East Road, Yonghe D...",24.995694,121.529712,351617.954635,2.765275e+06,5840.376700,0,2044.201559
6,"No. 124, Jingfu Street, Wenshan District, Taip...",24.995752,121.535656,352217.954635,2.765275e+06,5992.495307,1,1937.196363
7,"No. 213, Jianba Road, Zhonghe District, New Ta...",24.999998,121.491021,347717.954635,2.765795e+06,5855.766389,0,2292.832949
8,"No. 87號, Liancheng Road, Zhonghe District, New...",25.000058,121.496965,348317.954635,2.765795e+06,5604.462508,2,1819.175812
9,"No. 167, Lane 165, Zhonghe Road, Zhonghe Distr...",25.000118,121.502909,348917.954635,2.765795e+06,5408.326913,4,1307.398820


In [24]:
print('Average distance to closest Italian restaurant from each area center:', df_locations['Distance to Italian restaurant'].mean())

Average distance to closest Italian restaurant from each area center: 1579.2245937379353


OK, so **on average Italian restaurant can be found within ~1.6km** from every area center candidate. That's not close, so we need to filter our areas carefully!

Let's create a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. Also, let's show **borders of Taipei boroughs** on our map and a few circles indicating distance of 1km, 2km and 3km from Taipei Main Station.

In [42]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

italian_latlons = [[res[2], res[3]] for res in italian_restaurants.values()]

In [109]:
from folium import plugins
from folium.plugins import HeatMap

map_city = folium.Map(location=city_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_city) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_city)
folium.Marker(city_center).add_to(map_city)
folium.Circle(city_center, radius=1000, fill=False, color='white').add_to(map_city)
folium.Circle(city_center, radius=2000, fill=False, color='white').add_to(map_city)
folium.Circle(city_center, radius=3000, fill=False, color='white').add_to(map_city)
map_city

<img src='./9-3-2-3.jpg'>

Looks like a few pockets of high restaurant density closest to city center can be found **south, south-east and east from Taipei Main Station**. 


Let's create another heatmap map showing **heatmap/density of Italian restaurants** only.

In [66]:
map_city = folium.Map(location=city_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_city) #cartodbpositron cartodbdark_matter
HeatMap(italian_latlons).add_to(map_city)
folium.Marker(city_center).add_to(map_city)
folium.Circle(city_center, radius=1000, fill=False, color='white').add_to(map_city)
folium.Circle(city_center, radius=2000, fill=False, color='white').add_to(map_city)
folium.Circle(city_center, radius=3000, fill=False, color='white').add_to(map_city)
map_city

<img src='./9-3-2-4.jpg'>

This map is not so 'hot'. Italian restaurants represent a subset of ~5% of all restaurants in Taipei. But it also indicates higher density of existing Italian restaurants southeast from Taipei Main Station.

Based on this we will now focus our analysis on areas *south-east from Taipei center* - we will move the center of our area of interest and reduce it's size to have a radius of **2km**. This places our location candidates mostly in **the Eastern District of Taipei**.


### The Eastern District of Taipei

**The Eastern District of Taipei (Chinese: 臺北東區)** refers to the newly developed area in eastern Taipei, Taiwan. In its broadest sense, the Eastern District of Taipei is the whole region east of Fuxing South Road. In general, however, the Eastern District refers to the area between Civic Boulevard and Xinyi Road, including most part of Daan District, Xinyi District and Songshan District, the administrative districts in eastern Taipei. 

With a plethora of business buildings, department stores and shopping districts, the Eastern District has now become one of the most cosmopolitan parts of Taipei. Taipei City Hall, Taipei 101 and Daan Forest Park are all located in this district.

In [123]:
# roi_x_min = city_center_x
# roi_y_max = city_center_y
# roi_width = 3000
# roi_height = 3000
roi_center_x = city_center_x + 3500
roi_center_y = city_center_y - 1000
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_city = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_city)
folium.Marker(city_center).add_to(map_city)
folium.Circle(roi_center, radius=2000, color='white', fill=True, fill_opacity=0.4).add_to(map_city)
map_city

<img src='./9-3-2-5.jpg'>

Not bad - this nicely covers all the pockets of high restaurant density in the Eastern District closest to Taipei center.

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 100m appart).

In [133]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2001):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

1369 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest Italian restaurant**.

In [134]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_italian_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, italian_restaurants)
    roi_italian_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [135]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Italian restaurant':roi_italian_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Italian restaurant
0,25.020589,121.549730,353667.954635,2.768011e+06,5,499.889909
1,25.020598,121.550721,353767.954635,2.768011e+06,5,439.842154
2,25.020608,121.551712,353867.954635,2.768011e+06,6,396.273035
3,25.020617,121.552702,353967.954635,2.768011e+06,10,364.068878
4,25.020627,121.553693,354067.954635,2.768011e+06,10,302.633643
5,25.020637,121.554684,354167.954635,2.768011e+06,10,265.759469
6,25.021336,121.546253,353317.954635,2.768097e+06,1,554.755774
7,25.021346,121.547244,353417.954635,2.768097e+06,2,500.307898
8,25.021356,121.548234,353517.954635,2.768097e+06,6,453.148006
9,25.021366,121.549225,353617.954635,2.768097e+06,6,424.709593


OK. Let us now **filter** those locations: we're interested only in **locations with no more than two restaurants in radius of 250 meters**, and **no Italian restaurants in radius of 400 meters**.

In [136]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to Italian restaurant']>=400)
print('Locations with no Italian restaurants within 400m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two restaurants nearby: 211
Locations with no Italian restaurants within 400m: 519
Locations with both conditions met: 127


Let's see how this looks on a map.

In [137]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_city = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_city)
HeatMap(restaurant_latlons).add_to(map_city)
folium.Circle(roi_center, radius=2000, color='white', fill=True, fill_opacity=0.6).add_to(map_city)
folium.Marker(city_center).add_to(map_city)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_city) 
map_city

<img src='./9-3-2-6.jpg'>

Looking good. We now have a bunch of locations fairly close to the Eastern District of Taipei, and we know that each of those locations has no more than two restaurants in radius of 250m, and no Italian restaurant closer than 400m. Any of those locations is a potential candidate for a new Italian restaurant, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [138]:
map_city = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_city)
folium.Marker(city_center).add_to(map_city)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_city)
map_city

<img src='./9-3-2-7.jpg'>

Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and *no* Italian restaurants at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [139]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_city = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_city)
HeatMap(restaurant_latlons).add_to(map_city)
folium.Circle(roi_center, radius=2000, color='white', fill=True, fill_opacity=0.4).add_to(map_city)
folium.Marker(city_center).add_to(map_city)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_city) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_city)
map_city

<img src='./9-3-2-8.jpg'>

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [143]:
map_city = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(city_center).add_to(map_city)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_city)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_city)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_city) 
map_city

<img src='./9-3-2-9.jpg'>

Let's zoom in on candidate areas in **Minsheng Community (Chinese:民生社區)**:

In [156]:
map_city = folium.Map(location=[25.058661, 121.562673], zoom_start=15)
icon=folium.Icon(color='red', icon='info-sign')
folium.Marker([25.058661, 121.562673],popup='民生社區',icon=icon).add_to(map_city)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_city) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_city)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_city)
map_city

<img src='./9-3-2-10.jpg'>

...and candidate areas in **Daan District (Chinese:大安區)**:

In [185]:
map_city = folium.Map(location=[25.026460, 121.535560], zoom_start=15)
icon=folium.Icon(color='green', icon='info-sign')
folium.Marker([25.026460, 121.535560],popup='大安區',icon=icon).add_to(map_city)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_city) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_city)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_city)
map_city

<img src='./9-3-2-11.jpg'>

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [222]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', Taiwan', '')
    addr = addr.replace('’', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, city_center_x, city_center_y)
    print('{}{} => {:.1f}km from Taipei Main Station'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

105, Taipei City, Songshan District, 5F, No 331 Fusing N. Rd., Taipei 105 => 3.8km from Taipei Main Station
No. 152-3, Section 3, Xinyi Road, Daan District, Taipei City 106 => 2.8km from Taipei Main Station
No. 11, Alley 201, Lane 103, Xin'an Street, Xinyi District, Taipei City 110 => 4.9km from Taipei Main Station
No. 47, Lane 10, Section 2, Bade Road, Daan District, Taipei City 106 => 2.0km from Taipei Main Station
No. 22, Lane 46, Guangfu South Road, Songshan District, Taipei City 105 => 3.9km from Taipei Main Station
No. 19-3, Lane 56, Wolong Street, Daan District, Taipei City 106 => 4.0km from Taipei Main Station
Daan Forest Park, No. 1號, Section 2, Xinsheng South Road, Daan District, Taipei City 106 => 2.7km from Taipei Main Station
No. 35, Jianguo Elevated Road, Daan District, Taipei City 106 => 2.6km from Taipei Main Station
No. 2, Alley 3, Lane 118, Section 3, Ren'ai Road, Daan District, Taipei City 106 => 2.6km f

This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of restaurants and no Italian restaurants nearby, all zones being fairly close to city center (all less than 4km from Taipei Main Station, and about half of those less than 2km from Taipei Main Station). Although zones are shown on map with a radius of ~500 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. Most of the zones are located in Daan District, which we have identified as interesting due to being popular with tourists, fairly close to city center and well connected by public transport.

In [247]:
map_city = folium.Map(location=roi_center, zoom_start=14)
folium.Circle(city_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_city)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_city)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    icon=folium.Icon(color='orange', icon='info-sign')
    folium.Marker([lonlat[1], lonlat[0]], popup=str(addr),icon=icon).add_to(map_city) 
map_city


<img src='./9-3-2-12.jpg'>

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Taipei (~2000 in our initial area of interest which was 12x12km around Taipei Main Station), there are pockets of low restaurant density fairly close to city center. Highest concentration of restaurants was detected south and east from Taipei Main Station, so we focused our attention to areas south, south-east and east, corresponding to the Eastern District of Taipei. 

Another borough was identified as potentially interesting (Minsheng Community, north-east from Taipei Main Station), but our attention was focused on Daan District which offer a combination of popularity among tourists, closeness to city center, strong socio-economic dynamics *and* a number of pockets of low restaurant density.

After directing our attention to this more narrow area of interest (covering approx. 5x5km south-east from Taipei Main Station) we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two restaurants in radius of 250m and those with an Italian restaurant closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 15 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Italian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Taipei center but not crowded with existing restaurants (particularly Italian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Taipei areas close to center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis (The Eastern District of Taipei), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.